In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import kale.workflow_objects
import kale.workflow_widgets

# Bash/Python Integrated Workflow
I'm using Python's `subprocess` module to execute bash tasks via parsl's Python tasks, rather than using parsl's native bash tasks, which I expected to be using.

I did this to be able to use `AppFuture`s, which don't seem to be available with bash tasks. It seems that bash tasks in parsl only support `DataFuture` dependencies, so the absence of a file is the only thing that can block a parsl bash task from executing, whereas I want to wait for the previous function to finish, whether or not a new file is created at the end.

I may be misunderstanding parsl's bash tasks and `DataFuture`s.

In [ ]:
def hello():
    import time
    print("Hello from Python!")
    time.sleep(1)
    return 'hi'
def bye():
    import time
    print("Goodbye from Python!")
    time.sleep(1)
    return 'bye'
def theysaid(num, msg):
    import time
    print("Task {} returned '{}'".format(num, msg))
    time.sleep(1)

In [ ]:
t0 = kale.workflow_objects.PythonFunctionTask(
    name='HelloPy',
    func=hello
)
t1 = kale.workflow_objects.CommandLineTask(
    name='HelloBash',
    command="echo 'Hello from Bash!' && sleep 1"
)
t2 = kale.workflow_objects.PythonFunctionTask(
    name='TheySaid0',
    func=theysaid,
    args=[0, t0]
)
t3 = kale.workflow_objects.PythonFunctionTask(
    name='TheySaid1',
    func=theysaid,
    args=[1, t1]
)
t4 = kale.workflow_objects.PythonFunctionTask(
    name='ByePy',
    func=bye
)

bp_wf = kale.workflow_objects.Workflow(name='bp_wf')

bp_wf.add_task(t0)
bp_wf.add_task(t1, dependencies=[t0])
bp_wf.add_task(t2, dependencies=[t0])
bp_wf.add_task(t3, dependencies=[t1])
bp_wf.add_task(t4, dependencies=[t1, t2, t3])

In [ ]:
wpw = kale.workflow_widgets.WorkerPoolWidget()
wpw

In [ ]:
kale.workflow_widgets.WorkflowWidget(bp_wf, wpw)